In [9]:
from llama3.utils import (
    get_llama3,
    LlamaIndexCustomLLM
)
from llama3.config import (
    MODELS_DIR,
    DATA_DIR
)
import os

In [2]:
adapter = "Meta-Llama-3-8B-Instruct_ft_pop_kar_1_ep_1split"
llm_agrillama, tokenizer_agrillama = get_llama3(adapter_path=os.path.join(MODELS_DIR, adapter))
llm_base, tokenizer_base = get_llama3()

/data2/home/sahilkamble/anaconda3/envs/project/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/data2/home/sahilkamble/anaconda3/envs/project/lib/python3.12/site-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/data2/home/sahilkamble/anaconda3/envs/project/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/data2/home/sahilkamble/anaconda3/envs/project/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
agrillama3 = LlamaIndexCustomLLM(llm_agrillama, tokenizer_agrillama)
llama3 = LlamaIndexCustomLLM(llm_base, tokenizer_base)

In [4]:
text ='''<|begin_of_text|><|start_header_id|>system<|end_header_id|> 
You are helpful agricuture advisor. Answer the queries factually and truthfully in concise manner.
Query: A farmer from Banglore district of KARNATAKA asked about control of t-flies in guava.
Answer: spray carbaryl 4g per liter
Query: A farmer from BELGAUM district of KARNATAKA asked about fertiliser recomendation for Coconut.
Answer: Apply 1:1:1 kg/plant Ammonium sulphate:SSP:MOP twice in a year in may-june another in sept-octo
Query: A farmer from BELGAUM district of KARNATAKA asked about how to safely store jowar
Answer: dry the seeds to safe moisture level and fumigate with alluminium phosphide if storage pests are found
Query: A farmer from BIDAR district of KARNATAKA asked about variety of rose.
Answer: sweet heart  baby rotras elon karona
<|eot_id|><|start_header_id|>user<|end_header_id|>
A farmer from BELGAUM district of KARNATAKA asked about planting season for coconut.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>'''


print(print(agrillama3.complete(text).text))



In Karnataka, the ideal planting season for coconut is from March to May and September to November. These periods coincide with the onset of the southwest monsoon and the northeast monsoon, respectively, which provide adequate rainfall and moisture for the coconut seedlings.
None


In [5]:
import pandas as pd

In [10]:
data = pd.read_csv(os.path.join(DATA_DIR, "clean_karna_kcc.csv"))

In [26]:
data.columns

Index(['Index  ', 'BlockName          ', 'Category ', 'Year ', 'Month ',
       'Day ', 'Crop                 ', 'DistrictName ',
       'QueryType                       ', 'Season ', 'Sector           ',
       'StateName ',
       'QueryText                                                                             ',
       'KccAns'],
      dtype='object')

In [18]:
data = data.iloc[:100]

In [11]:
from tqdm import tqdm

In [27]:
data.columns = data.columns.str.strip() 

In [37]:
answers = []
for _, row in tqdm(data.iterrows(), total=len(data)):
    # text =f'''<|begin_of_text|><|start_header_id|>system<|end_header_id|> 
    # You are helpful agricuture advisor. Answer the queries factually and truthfully in a concise manner in few words.
    # <|eot_id|><|start_header_id|>user<|end_header_id|>
    # A farmer from {row['DistrictName']} district of {row['StateName']} asked about {row['QueryText']}.
    # <|eot_id|><|start_header_id|>assistant<|end_header_id|>'''
    text =f'''<|begin_of_text|><|start_header_id|>system<|end_header_id|> 
    You are helpful agricuture advisor. Answer the queries factually and truthfully in a concise manner in few words.
    Query: A farmer from Banglore district of KARNATAKA asked about control of t-flies in guava.
    Answer: spray carbaryl 4g per liter
    Query: A farmer from BELGAUM district of KARNATAKA asked about fertiliser recomendation for Coconut.
    Answer: Apply 1:1:1 kg/plant Ammonium sulphate:SSP:MOP twice in a year in may-june another in sept-octo
    Query: A farmer from BELGAUM district of KARNATAKA asked about how to safely store jowar
    Answer: dry the seeds to safe moisture level and fumigate with alluminium phosphide if storage pests are found
    Query: A farmer from BIDAR district of KARNATAKA asked about variety of rose.
    Answer: sweet heart  baby rotras elon karona
    <|eot_id|><|start_header_id|>user<|end_header_id|>
    A farmer from {row['DistrictName']} district of {row['StateName']} asked about {row['QueryText']}.
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>'''
    answers.append(llama3.complete(text).text)

100%|██████████| 100/100 [08:05<00:00,  4.85s/it]


In [38]:
answers_df = pd.DataFrame({'Answer': answers})
# Write answers to a CSV file
answers_df.to_csv("llama_fewshot_ans.csv", index=False)